# Introduction

## Background Information:

In 2016, Cyclistic launched a successful bike-share offering. Since then, the program has grown to a fleet of 5,824 bicycles that are geotracked and locked into a network of 692 stations across Chicago. The bikes can be unlocked from one station and returned to any other station in the system anytime.

Until now, Cyclistic’s marketing strategy relied on building general awareness and appealing to broad consumer segments. One approach that helped make these things possible was the flexibility of its pricing plans: single-ride passes, full-day passes, and annual memberships. Customers who purchase single-ride or full-day passes are referred to as casual riders. Customers who purchase annual memberships are Cyclistic members.

## The Problem:

Cyclistic’s finance analysts have concluded that annual members are much more profitable than casual riders. Although the pricing flexibility helps Cyclistic attract more customers, __the executive team believes that maximizing the number of annual members will be key to future growth.__ Rather than creating a marketing campaign that targets all-new customers, there is a very good chance to convert casual riders into members with targeted marketing.

## Objective: 

- To better understand how annual members and casual riders differ.

- To determine why casual riders would buy a membership.


__The insights generated from this analysis would allow Cyclistic to design marketing strategies aimed at converting casual riders into annual members.__

![image info](./images/bike_sharing.jpeg)

### Load Libraries

In [1]:
library("tidyverse")

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


### Load Data

We will use data from Jun-2020 to May-2021 for this analysis. Data is provided by month and we will merge them into one dataframe.

In [2]:
# load individual datasets

tripdata_2020_06 <- read.csv("data/csv/202006-divvy-tripdata.csv")
tripdata_2020_07 <- read.csv("data/csv/202007-divvy-tripdata.csv")
tripdata_2020_08 <- read.csv("data/csv/202008-divvy-tripdata.csv")
tripdata_2020_09 <- read.csv("data/csv/202009-divvy-tripdata.csv")
tripdata_2020_10 <- read.csv("data/csv/202010-divvy-tripdata.csv")
tripdata_2020_11 <- read.csv("data/csv/202011-divvy-tripdata.csv")
tripdata_2020_12 <- read.csv("data/csv/202012-divvy-tripdata.csv")
tripdata_2021_01 <- read.csv("data/csv/202101-divvy-tripdata.csv")
tripdata_2021_02 <- read.csv("data/csv/202102-divvy-tripdata.csv")
tripdata_2021_03 <- read.csv("data/csv/202103-divvy-tripdata.csv")
tripdata_2021_04 <- read.csv("data/csv/202103-divvy-tripdata.csv")
tripdata_2021_05 <- read.csv("data/csv/202103-divvy-tripdata.csv")

In [3]:
# display sample data

head(tripdata_2020_06)

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
8CD5DE2C2B6C4CFC,docked_bike,2020-06-13 23:24:48,2020-06-13 23:36:55,Wilton Ave & Belmont Ave,117,Damen Ave & Clybourn Ave,163,41.94018,-87.65304,41.93193,-87.67786,casual
9A191EB2C751D85D,docked_bike,2020-06-26 07:26:10,2020-06-26 07:31:58,Federal St & Polk St,41,Daley Center Plaza,81,41.87208,-87.62954,41.88424,-87.62963,member
F37D14B0B5659BCF,docked_bike,2020-06-23 17:12:41,2020-06-23 17:21:14,Daley Center Plaza,81,State St & Harrison St,5,41.88424,-87.62963,41.87405,-87.62772,member
C41237B506E85FA1,docked_bike,2020-06-20 01:09:35,2020-06-20 01:28:24,Broadway & Cornelia Ave,303,Broadway & Berwyn Ave,294,41.94553,-87.64644,41.97835,-87.65975,casual
4B51B3B0BDA7787C,docked_bike,2020-06-25 16:59:25,2020-06-25 17:08:48,Sheffield Ave & Webster Ave,327,Wilton Ave & Belmont Ave,117,41.92154,-87.65382,41.94018,-87.65304,casual
D50DF288196B53BE,docked_bike,2020-06-17 18:07:18,2020-06-17 18:18:14,Sheffield Ave & Webster Ave,327,Wilton Ave & Belmont Ave,117,41.92154,-87.65382,41.94018,-87.65304,casual


In [4]:
# merging into one dataframe

full_df <- rbind(tripdata_2020_06, 
                 tripdata_2020_07,
                 tripdata_2020_08, 
                 tripdata_2020_09,
                 tripdata_2020_10, 
                 tripdata_2020_11,
                 tripdata_2020_12,
                 tripdata_2021_01,
                 tripdata_2021_02,
                 tripdata_2021_03,
                 tripdata_2021_04,
                 tripdata_2021_05)

# get dimensions of resulting dataframe
print(dim(full_df))

[1] 3661690      13


### Data Quality Check

In [5]:
# check for duplicate rows

# get unique count of ride ID
print(n_distinct(full_df$ride_id))

[1] 3204489


There are over 400k duplicates in the merged dataframe. We will go ahead and remove them.

In [6]:
# drop duplicates
full_df <- full_df[!duplicated(full_df$ride_id), ]

# get dimensions of resulting dataframe
print(dim(full_df))

# get unique count of ride ID
print(n_distinct(full_df$ride_id))

[1] 3204489      13
[1] 3204489


In [7]:
# check data types

str(full_df)

'data.frame':	3204489 obs. of  13 variables:
 $ ride_id           : Factor w/ 3204489 levels "00000550C665101A",..: 189053 206525 326283 262994 100757 285656 29706 289700 211205 82745 ...
 $ rideable_type     : Factor w/ 3 levels "docked_bike",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ started_at        : Factor w/ 2777637 levels "2020-06-03 05:59:59",..: 99985 239542 208684 169915 232343 139910 225668 155403 290665 268915 ...
 $ ended_at          : Factor w/ 2765669 levels "2020-06-03 06:03:37",..: 99802 239021 207940 169557 231559 139367 225227 155086 290026 267858 ...
 $ start_station_name: Factor w/ 707 levels "2112 W Peterson Ave",..: 598 227 166 49 481 481 227 482 73 398 ...
 $ start_station_id  : chr  "117" "41" "81" "303" ...
 $ end_station_name  : Factor w/ 707 levels "","2112 W Peterson Ave",..: 172 167 525 49 602 602 167 50 240 577 ...
 $ end_station_id    : chr  "163" "81" "5" "294" ...
 $ start_lat         : num  41.9 41.9 41.9 41.9 41.9 ...
 $ start_lng         : num  -87.7 -87.6 -87

In [8]:
# convert started_at and ended_at to datetime format

full_df$started_at <- as.POSIXct(full_df$started_at, "%Y-%m-%d %H:%M:%S")
full_df$ended_at <- as.POSIXct(full_df$ended_at, "%Y-%m-%d %H:%M:%S")

In [9]:
# check resulting data type

str(full_df$started_at)
str(full_df$ended_at)

 POSIXct[1:3204489], format: "2020-06-13 23:24:48" "2020-06-26 07:26:10" "2020-06-23 17:12:41" ...
 POSIXct[1:3204489], format: "2020-06-13 23:36:55" "2020-06-26 07:31:58" "2020-06-23 17:21:14" ...


In [10]:
# check for null values

map(full_df, ~sum(is.na(.)))

$ride_id
[1] 0

$rideable_type
[1] 0

$started_at
[1] 0

$ended_at
[1] 0

$start_station_name
[1] 0

$start_station_id
[1] 83583

$end_station_name
[1] 0

$end_station_id
[1] 97684

$start_lat
[1] 0

$start_lng
[1] 0

$end_lat
[1] 4317

$end_lng
[1] 4317

$member_casual
[1] 0

There are over 90k rows with null values in the end_station_id column. We will go ahead and drop them.

In [11]:
# drop null values

full_df_clean <- drop_na(full_df)

# get dimensions of resulting dataframe
print(dim(full_df_clean))

[1] 3069676      13


### Data Mining

We can extract individual date parts which can be useful for our analysis.

In [12]:
# extract date
full_df_clean$date <- as.Date(full_df_clean$started_at) 

# extract month
full_df_clean$month <- format(as.Date(full_df_clean$date), "%m")

# extract day
full_df_clean$day <- format(as.Date(full_df_clean$date), "%d")

# extract year
full_df_clean$year <- format(as.Date(full_df_clean$date), "%Y")

# extract day of week
full_df_clean$day_of_week <- format(as.Date(full_df_clean$date), "%A")

In [13]:
head(full_df_clean)

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,month,day,year,day_of_week
8CD5DE2C2B6C4CFC,docked_bike,2020-06-13 23:24:48,2020-06-13 23:36:55,Wilton Ave & Belmont Ave,117,Damen Ave & Clybourn Ave,163,41.94018,-87.65304,41.93193,-87.67786,casual,2020-06-13,06,13,2020,Saturday
9A191EB2C751D85D,docked_bike,2020-06-26 07:26:10,2020-06-26 07:31:58,Federal St & Polk St,41,Daley Center Plaza,81,41.87208,-87.62954,41.88424,-87.62963,member,2020-06-26,06,26,2020,Friday
F37D14B0B5659BCF,docked_bike,2020-06-23 17:12:41,2020-06-23 17:21:14,Daley Center Plaza,81,State St & Harrison St,5,41.88424,-87.62963,41.87405,-87.62772,member,2020-06-23,06,23,2020,Tuesday
C41237B506E85FA1,docked_bike,2020-06-20 01:09:35,2020-06-20 01:28:24,Broadway & Cornelia Ave,303,Broadway & Berwyn Ave,294,41.94553,-87.64644,41.97835,-87.65975,casual,2020-06-20,06,20,2020,Saturday
4B51B3B0BDA7787C,docked_bike,2020-06-25 16:59:25,2020-06-25 17:08:48,Sheffield Ave & Webster Ave,327,Wilton Ave & Belmont Ave,117,41.92154,-87.65382,41.94018,-87.65304,casual,2020-06-25,06,25,2020,Thursday
D50DF288196B53BE,docked_bike,2020-06-17 18:07:18,2020-06-17 18:18:14,Sheffield Ave & Webster Ave,327,Wilton Ave & Belmont Ave,117,41.92154,-87.65382,41.94018,-87.65304,casual,2020-06-17,06,17,2020,Wednesday


We can also extract useful information from the bike rides (duration, distance etc).

In [14]:
# create new column ride_duration_mins -> total ride duration in mins

full_df_clean <- full_df_clean %>%
    mutate(ride_duration_mins = as.numeric(full_df_clean$ended_at - full_df_clean$started_at) / 60)

In [15]:
summary(full_df_clean$ride_duration_mins)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
-29049.97      7.83     14.30     25.63     26.23  54283.35 

It appears that there are rides where the end time is before the start time. We will treat this as a data entry error and will go ahead to drop them from our dataframe.

In [16]:
# check rides with negative ride duration

filter(full_df_clean, ride_duration_mins < 0)

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,month,day,year,day_of_week,ride_duration_mins
2344EF0C012BEDC7,docked_bike,2020-06-30 12:49:45,2020-06-30 12:49:24,Lakefront Trail & Bryn Mawr Ave,459,Lake Shore Dr & Wellington Ave,157,41.98404,-87.65231,41.93669,-87.63683,member,2020-06-30,06,30,2020,Tuesday,-0.35000000
870547A756460EFB,docked_bike,2020-06-07 16:02:33,2020-06-07 16:02:26,Wilton Ave & Belmont Ave,117,Rush St & Superior St,161,41.94018,-87.65304,41.89576,-87.62591,casual,2020-06-07,06,07,2020,Sunday,-0.11666667
B5D8BCC20CAB2CBF,docked_bike,2020-06-16 12:37:26,2020-06-16 12:37:24,Mies van der Rohe Way & Chicago Ave,173,Sheffield Ave & Willow St,93,41.89691,-87.62174,41.91369,-87.65286,member,2020-06-16,06,16,2020,Tuesday,-0.03333333
D4D3282374FB5B2A,docked_bike,2020-06-30 18:55:29,2020-06-30 18:55:26,Sheffield Ave & Kingsbury St,20,Greenview Ave & Diversey Pkwy,319,41.91052,-87.65311,41.93259,-87.66594,casual,2020-06-30,06,30,2020,Tuesday,-0.05000000
D7C40BD25CA7B392,docked_bike,2020-06-28 17:37:54,2020-06-28 17:37:53,Montrose Harbor,249,Lakefront Trail & Bryn Mawr Ave,459,41.96398,-87.63818,41.98404,-87.65231,member,2020-06-28,06,28,2020,Sunday,-0.01666667
107186E0900BFB98,docked_bike,2020-06-16 18:22:19,2020-06-16 18:22:15,Loomis St & Archer Ave,366,Wood St & 35th St,369,41.84163,-87.65743,41.83010,-87.67030,member,2020-06-16,06,16,2020,Tuesday,-0.06666667
01AF02960F449E86,docked_bike,2020-06-16 18:53:43,2020-06-16 18:53:26,Wood St & 35th St,369,Wood St & 35th St,369,41.83010,-87.67030,41.83010,-87.67030,member,2020-06-16,06,16,2020,Tuesday,-0.28333333
F1BB6AC7855F4DC7,docked_bike,2020-06-28 13:33:34,2020-06-28 13:33:27,Larrabee St & Webster Ave,144,Larrabee St & Webster Ave,144,41.92182,-87.64414,41.92182,-87.64414,casual,2020-06-28,06,28,2020,Sunday,-0.11666667
A9FF70C3F11A7BF1,docked_bike,2020-06-25 17:41:09,2020-06-25 17:41:02,May St & Taylor St,22,Mies van der Rohe Way & Chicago Ave,173,41.86948,-87.65549,41.89691,-87.62174,casual,2020-06-25,06,25,2020,Thursday,-0.11666667
EE4E85BDD80429B0,docked_bike,2020-06-11 17:47:49,2020-06-11 17:47:47,Michigan Ave & Lake St,52,Larrabee St & Webster Ave,144,41.88602,-87.62412,41.92182,-87.64414,casual,2020-06-11,06,11,2020,Thursday,-0.03333333


In [17]:
# number of rows with negative ride duration

dim(filter(full_df_clean, ride_duration_mins < 0))[1]

[1] 10088

In [18]:
# drop rows with negative ride duration
full_df_clean <- subset(full_df_clean, ride_duration_mins > 0)

# get dimensions of resulting dataframe
print(dim(full_df_clean))

[1] 3059297      19
